In [2]:
include("../../L-BFGS/OracleFunction.jl")
include("../../L-BFGS/LBFGS.jl")
include("../../utilities/genFunc.jl")
using .LBFGS
using .OracleFunction
using LinearAlgebra, BenchmarkTools, CSV, DataFrames

baseDir = joinpath("../", "results/LBFGS/well_conditioned/")
mkpath(baseDir);

In [3]:
# parameters for exact LS vs AWLS vs quadratic function 
λ        = 10^-4
ϵ        = 10^-14
memsize  = 7
maxIters = 1:200
m        = 1000
n        = 20
num_trials = 20

gf = genFunc(:exactRandDataset, λ=λ, m=m, n=n)
non_quadratic = OracleF(ones(size(gf[:X_hat], 2)),
        (x) -> norm(gf[:X_hat] * x - gf[:y_hat]),
        (x) -> inv(norm(gf[:X_hat] * x - gf[:y_hat])) * gf[:X_hat]' * (gf[:X_hat] * x - gf[:y_hat])
    )
quadratic = OracleF(ones(size(gf[:X_hat], 2)),
        (x) -> norm(gf[:X_hat] * x - gf[:y_hat]) ^ 2,
        (x) -> 2 * gf[:X_hat]' * (gf[:X_hat] * x - gf[:y_hat])
    )
ls = LeastSquaresF(gf)

LeastSquaresF{Float64, Main.OracleFunction.var"#f#1"{Matrix{Float64}, Vector{Float64}}, Main.OracleFunction.var"#df#2"{Matrix{Float64}, Vector{Float64}}}(OracleF{Float64, Main.OracleFunction.var"#f#1"{Matrix{Float64}, Vector{Float64}}, Main.OracleFunction.var"#df#2"{Matrix{Float64}, Vector{Float64}}}([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], Main.OracleFunction.var"#f#1"{Matrix{Float64}, Vector{Float64}}([-0.08250861069928206 0.17852719210948242 … 0.19042005909630877 -0.507902809042877; -0.5572519727292988 0.715127783454649 … -0.9011645602213216 0.038321040360024705; … ; 0.0 0.0 … 0.0001 0.0; 0.0 0.0 … 0.0 0.0001], [-7.9909737306123745, 14.69538119428254, 2.4593509793825494, -3.953587686159141, 4.252647143131211, -4.911100267400292, 18.00878115460575, -11.083562830390674, 7.4308455637609505, 1.3130519694521448  …  6.998564939162414e-5, 3.056340701582987e-5, -2.4008979560648935e-5, -9.410492147107272e-5, 2.467191606780468e-5

## Exact Line Search vs AWLS vs AWLS non quadratic

In [4]:
### residual, relative error and norm of gradient with respect to iterations with exact line search
using Statistics, CSV

# Preallocate arrays
num_iterations = length(maxIters)

gradients = zeros(num_trials, num_iterations)
residuals = zeros(num_trials, num_iterations)
relative_errors = zeros(num_trials, num_iterations)

for trial in 1:num_trials

    gf = genFunc(:exactRandDataset, λ=λ, m=m, n=n)
    ls = LeastSquaresF(gf)

    for (i, maxIter) in enumerate(maxIters)
        t = LimitedMemoryBFGS(ls, ϵ=ϵ, MaxEvaluations=maxIter, m=memsize)

        relative_errors[trial, i] = norm(t[:x] - gf[:w_star]) / norm(gf[:w_star])
        residuals[trial, i] = norm(gf[:X_hat] * t[:x] - gf[:y_hat]) / norm(gf[:y_hat])
        gradients[trial, i] = norm(t[:grad])
    end

    println("Done trial ", trial)
    
end

# Calculate mean and standard deviation
mean_relative = mean(relative_errors, dims=1)'
std_relative = std(relative_errors, dims=1)'
mean_residual = mean(residuals, dims=1)'
std_residual = std(residuals, dims=1)'
mean_gradient = mean(gradients, dims=1)'
std_gradient = std(gradients, dims=1)'


# Write results to CSV
outputvsc = joinpath(baseDir, "statisticsLBFGS-iterations-m" * string(m) * "n" * string(n) * "--error-norm.csv");

accData = Dict(
    :lambda        => Array{Float64}(undef, 0),
    :epsilon       => Array{Float64}(undef, 0),
    :memsize       => Array{Int64}(undef, 0),
    :maxiterations => Array{Int64}(undef, 0),
    :mean_relative => Array{Float64}(undef, 0),
    :std_relative  => Array{Float64}(undef, 0),
    :mean_residual => Array{Float64}(undef, 0),
    :std_residual  => Array{Float64}(undef, 0),
    :mean_gradient => Array{Float64}(undef, 0),
    :std_gradient  => Array{Float64}(undef, 0)
)

# create dataframe with columns from arrays
for maxIter ∈ maxIters
    push!(accData[:lambda], λ)
    push!(accData[:epsilon], ϵ)
    push!(accData[:memsize], memsize)
    push!(accData[:maxiterations], maxIter)
    push!(accData[:mean_relative], mean_relative[maxIter])
    push!(accData[:std_relative], std_relative[maxIter])
    push!(accData[:mean_residual], mean_residual[maxIter])
    push!(accData[:std_residual], std_residual[maxIter])
    push!(accData[:mean_gradient], mean_gradient[maxIter])
    push!(accData[:std_gradient], std_gradient[maxIter])

end


CSV.write(outputvsc, accData)


Done trial 1
Done trial 2
Done trial 3
Done trial 4
Done trial 5
Done trial 6
Done trial 7
Done trial 8
Done trial 9
Done trial 10
Done trial 11
Done trial 12
Done trial 13
Done trial 14
Done trial 15
Done trial 16
Done trial 17
Done trial 18
Done trial 19
Done trial 20


"../results/LBFGS/well_conditioned/statisticsLBFGS-iterations-m1000n20--error-norm.csv"

In [5]:
using Statistics, CSV

# Preallocate arrays
num_iterations = length(maxIters)


gradients = zeros(num_trials, num_iterations)
residuals = zeros(num_trials, num_iterations)
relative_errors = zeros(num_trials, num_iterations)


for trial in 1:num_trials
    gf = genFunc(:exactRandDataset, λ=λ, m=m, n=n)
    non_quadratic = OracleF(ones(size(gf[:X_hat], 2)),
                            (x) -> norm(gf[:X_hat] * x - gf[:y_hat]),
                            (x) -> inv(norm(gf[:X_hat] * x - gf[:y_hat])) * gf[:X_hat]' * (gf[:X_hat] * x - gf[:y_hat]))

    for (i, maxIter) in enumerate(maxIters)
        t = LimitedMemoryBFGS(non_quadratic, ϵ=ϵ, MaxEvaluations=maxIter, m=memsize)

        relative_errors[trial, i] = norm(t[:x] - gf[:w_star]) / norm(gf[:w_star])
        residuals[trial, i] = norm(gf[:X_hat] * t[:x] - gf[:y_hat]) / norm(gf[:y_hat])
        gradients[trial, i] = norm(t[:grad])
    end

    println("Done trial ", trial)
    
end

mean_relative = mean(relative_errors, dims=1)'
std_relative = std(relative_errors, dims=1)'
mean_residual = mean(residuals, dims=1)'
std_residual = std(residuals, dims=1)'
mean_gradient = mean(gradients, dims=1)'
std_gradient = std(gradients, dims=1)'

# Write results to CSV
outputvsc = joinpath(baseDir, "statisticsLBFGS-AWLS-iterations-m" * string(m) * "n" * string(n) * "--error-norm.csv")

accData = Dict(
    :lambda        => Array{Float64}(undef, 0),
    :epsilon       => Array{Float64}(undef, 0),
    :memsize       => Array{Int64}(undef, 0),
    :maxiterations => Array{Int64}(undef, 0),
    :mean_relative => Array{Float64}(undef, 0),
    :std_relative  => Array{Float64}(undef, 0),
    :mean_residual => Array{Float64}(undef, 0),
    :std_residual  => Array{Float64}(undef, 0),
    :mean_gradient => Array{Float64}(undef, 0),
    :std_gradient  => Array{Float64}(undef, 0)
)


for maxIter ∈ maxIters
    push!(accData[:lambda], λ)
    push!(accData[:epsilon], ϵ)
    push!(accData[:memsize], memsize)
    push!(accData[:maxiterations], maxIter)
    push!(accData[:mean_relative], mean_relative[maxIter])
    push!(accData[:std_relative], std_relative[maxIter])
    push!(accData[:mean_residual], mean_residual[maxIter])
    push!(accData[:std_residual], std_residual[maxIter])
    push!(accData[:mean_gradient], mean_gradient[maxIter])
    push!(accData[:std_gradient], std_gradient[maxIter])

end


CSV.write(outputvsc, accData)

Done trial 1
Done trial 2
Done trial 3
Done trial 4
Done trial 5
Done trial 6
Done trial 7
Done trial 8
Done trial 9
Done trial 10
Done trial 11
Done trial 12
Done trial 13
Done trial 14
Done trial 15
Done trial 16
Done trial 17
Done trial 18
Done trial 19
Done trial 20


"../results/LBFGS/well_conditioned/statisticsLBFGS-AWLS-iterations-m1000n20--error-norm.csv"

In [10]:
### error and norm of gradient with respect to iterations with AWLS and quadratic function
# output csv
using Statistics, CSV

maxIters = 1:400
num_trials = 5
# Preallocate arrays
num_iterations = length(maxIters)

gradients = zeros(num_trials, num_iterations)
residuals = zeros(num_trials, num_iterations)
relative_errors = zeros(num_trials, num_iterations)
iterations = zeros(num_trials, num_iterations)

for trial in 1:num_trials
    gf = genFunc(:exactRandDataset, λ=λ, m=m, n=n)
    quadratic = OracleF(ones(size(gf[:X_hat], 2)),
                        (x) -> norm(gf[:X_hat] * x - gf[:y_hat]) ^ 2,
                        (x) -> 2 * gf[:X_hat]' * (gf[:X_hat] * x - gf[:y_hat]))

    for (i, maxIter) in enumerate(maxIters)
        t = LimitedMemoryBFGS(quadratic, ϵ=ϵ, MaxEvaluations=maxIter, m=memsize)


        relative_errors[trial, i] = norm(t[:x] - gf[:w_star]) / norm(gf[:w_star])
        residuals[trial, i] = norm(gf[:X_hat] * t[:x] - gf[:y_hat]) / norm(gf[:y_hat])
        gradients[trial, i] = norm(t[:grad])
        iterations[trial, i] = i 
    end

    println("Done trial ", trial)
    
end


mean_relative = mean(relative_errors, dims=1)'
std_relative = std(relative_errors, dims=1)'
mean_residual = mean(residuals, dims=1)'
std_residual = std(residuals, dims=1)'
mean_gradient = mean(gradients, dims=1)'
std_gradient = std(gradients, dims=1)'

# Write results to CSV
outputvsc = joinpath(baseDir, "statisticsLBFGS-AWLS-quad-iterations-m" * string(m) * "n" * string(n) * "--error-norm.csv");

accData = Dict(
    :lambda        => Array{Float64}(undef, 0),
    :epsilon       => Array{Float64}(undef, 0),
    :memsize       => Array{Int64}(undef, 0),
    :maxiterations => Array{Int64}(undef, 0),
    :mean_relative => Array{Float64}(undef, 0),
    :std_relative  => Array{Float64}(undef, 0),
    :mean_residual => Array{Float64}(undef, 0),
    :std_residual  => Array{Float64}(undef, 0),
    :mean_gradient => Array{Float64}(undef, 0),
    :std_gradient  => Array{Float64}(undef, 0)
)


for maxIter ∈ maxIters
    push!(accData[:lambda], λ)
    push!(accData[:epsilon], ϵ)
    push!(accData[:memsize], memsize)
    push!(accData[:maxiterations], maxIter)
    push!(accData[:mean_relative], mean_relative[maxIter])
    push!(accData[:std_relative], std_relative[maxIter])
    push!(accData[:mean_residual], mean_residual[maxIter])
    push!(accData[:std_residual], std_residual[maxIter])
    push!(accData[:mean_gradient], mean_gradient[maxIter])
    push!(accData[:std_gradient], std_gradient[maxIter])

end

maxIters = 1:200
CSV.write(outputvsc, accData)

Done trial 1
Done trial 2
Done trial 3
Done trial 4
Done trial 5
Done trial 6
Done trial 7
Done trial 8
Done trial 9
Done trial 10
Done trial 11
Done trial 12
Done trial 13
Done trial 14
Done trial 15
Done trial 16
Done trial 17
Done trial 18
Done trial 19
Done trial 20


"../results/LBFGS/well_conditioned/statisticsLBFGS-AWLS-quad-iterations-m1000n20--error-norm.csv"

In [7]:
### relative error with repsepct to iterations and memory size
# output csv
outputvsc = joinpath(baseDir, "statisticsLBFGS-iterations-m" * string(m) * "n" * string(n) * "--memsize.csv");
accData = Dict(
    :memsize       => Array{Int64}(undef, 0),
    :iterations    => Array{Int64}(undef, 0),
    :maxiterations => Array{Int64}(undef, 0),
    :relative      => Array{Float64}(undef, 0),
    :residual      => Array{Float64}(undef, 0),
    :gradient      => Array{Float64}(undef, 0)
    )

memsizes = [1 3 5 7 9 11]

for memsize ∈ memsizes
    gf = genFunc(:exactRandDataset, λ=λ, m=m, n=n)
    ls = LeastSquaresF(gf)
    for maxIter ∈ maxIters

        t = LimitedMemoryBFGS(ls, ϵ=ϵ, MaxEvaluations=maxIter, m=memsize)

        relative_error = norm(t[:x] - gf[:w_star]) / norm(gf[:w_star])
        residual = norm(gf[:X_hat] * t[:x] - gf[:y_hat]) / norm(gf[:y_hat])

        iterations = maxIter - t[:RemainingEvaluations]

        push!(accData[:memsize], memsize)
        push!(accData[:iterations], iterations)
        push!(accData[:maxiterations], maxIter)
        push!(accData[:relative], relative_error)
        push!(accData[:residual], residual)
        push!(accData[:gradient], norm(t[:grad]))
        println("Done: maxIter " * string(iterations) * " -> error " * string(relative_error))
        flush(stdout)
    end
end

CSV.write(outputvsc, DataFrame(accData));

Done: maxIter 1 -> error 1.9723953204307332
Done: maxIter 3 -> error 1.939327681829027
Done: maxIter 3 -> error 1.939327681829027
Done: maxIter 5 -> error 1.938744601524112
Done: maxIter 5 -> error 1.938744601524112
Done: maxIter 7 -> error 1.9387301726555504
Done: maxIter 7 -> error 1.9387301726555504
Done: maxIter 9 -> error 1.938729924452633
Done: maxIter 9 -> error 1.938729924452633
Done: maxIter 11 -> error 1.9387299211343605
Done: maxIter 11 -> error 1.9387299211343605
Done: maxIter 13 -> error 1.9387299209849849
Done: maxIter 13 -> error 1.9387299209849849
Done: maxIter 15 -> error 1.9387299209279898
Done: maxIter 15 -> error 1.9387299209279898
Done: maxIter 17 -> error 1.9387299208702156
Done: maxIter 17 -> error 1.9387299208702156
Done: maxIter 19 -> error 1.9387299208084985
Done: maxIter 19 -> error 1.9387299208084985
Done: maxIter 21 -> error 1.938729920747817
Done: maxIter 21 -> error 1.938729920747817
Done: maxIter 23 -> error 1.9387299206796087
Done: maxIter 23 -> error 1